<a href="https://colab.research.google.com/github/kotaxtech/CUT/blob/main/CUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## セットアップ

In [ ]:
# Colab上のGPUの確認．
!nvidia-smi

Tue Feb  1 12:31:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# MyDriveに移動し，workspaceディレクトリを作成．
# Github(https://github.com/taesungp/contrastive-unpaired-translation)から該当リポジトリをクローン．

from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/workspace
%cd /content/drive/MyDrive/workspace
!git clone https://github.com/taesungp/contrastive-unpaired-translation.git CUT

In [ ]:
# CUTディレクトリに移動し，必要なライブラリをrequirements.txtのリストよりインストール．
# 評価指標となるFIDのライブラリは別途インストールする必要あり．

%cd /content/drive/MyDrive/workspace/CUT
!pip install -r requirements.txt
!pip install pytorch-fid

## データセット


CUTディレクトリ直下にあるdatasetsディレクトリに以下のように自前のデータセットを配置する．原画像からラベル画像へ変換したい場合，trainA/testAには原画像，trainB/testBにはラベル画像を配置する．拡張子は特に記述はないが，pngかjpgの統一を推奨．
```
./datasets
  └ image2label
        ├ trainA
        │  ├ image0000.png
        │  ├ image0001.png
        │  │    ・
        │  │    ・
        │  │    ・
        │  └ imagexxxx.png
        ├ trainB
        ├ testA
        └ testB
```

## 学習(Image2Label)

In [ ]:
# 以下のコードを実行すると，checkpointディレクトリが作成され，学習経過を示す画像と損失値，結合荷重が保存される．
# 各パラメータについてはoptionsディレクトリのbase_option.pyとtrain_option.pyを参照.

!python train.py \
  --dataroot ./datasets/image2label/ \
  --name img2label_CUT \
  --CUT_mode CUT \
  --n_epochs 50 \
  --n_epochs_decay 50 \
  --crop_size 256 \
  --batch_size 2 \
  --num_patches 64 \
  #--continue_train

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: ./datasets/soybean_img2label/ 	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: None                          
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost     

## テスト(Image2Label)

In [ ]:
# 以下のコードを実行すると，resultsディレクトリが作成され，出力画像が保存される．

!python test.py \
  --dataroot ./datasets/image2label \
  --name img2label_CUT \
  --CUT_mode CUT \
  --num_test 500 \
  --phase test \
  --epoch latest

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/soybean_img2label  	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
               easy_label: experiment_name               
                    epoch: 35                            	[default: latest]
                     eval: True                          	[default: False]
        flip_equivariance: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 3  

## FIDの計算


In [ ]:
# 以下のコードを実行すると，FIDが出力される．
# 第一引数にラベル画像(事前に用意したもの)，第二引数に生成画像(テストで生成された画像)のディレクトリを指定．

!python -m pytorch_fid ./datasets/image2label/testB/ ./results/image2label_CUT/test_latest/images/fake_B/

100% 4/4 [00:01<00:00,  2.16it/s]
100% 4/4 [00:01<00:00,  2.46it/s]
FID:  207.07649349433092


In [ ]:
# テストコードとFIDをスクリプトでたたく．

%% bash
touch ./result/image2label/FID.txt
echo "${epoch},${FID}\n" >> ./result/image2label/FID.txt

epochs = [25, 50, 75, 100]
for num in $(epochs)
do
  echo "${num}" >> ./result/image2label/FID.txt
  python --dataroot ./datasets/image2label --name img2label_CUT --CUT_mode CUT --num_test 500 --phase test --epoch ${num}
  python -m pytorch_fid ./datasets/image2label/testB/ ./results/img2gt_CUT/train_${num}/images/fake_B/ >> ./result/image2label/FID.txt
done

  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100% 1/1 [00:00<00:00,  1.03it/s]
100% 1/1 [00:00<00:00,  1.02it/s]
FID:  181.53864521865927
